In [ ]:
from azureml.core import Workspace

# set up workspace
ws = Workspace.from_config()

# Take a look at Workspace
ws.get_details()

# set up datastores
dstore = ws.get_default_datastore()

print('Workspace Name: ' + ws.name, 
      'Azure Region: ' + ws.location, 
      'Subscription Id: ' + ws.subscription_id, 
      'Resource Group: ' + ws.resource_group, 
      sep = '\n')

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, 'training-pipeline')

print('Experiment name: ' + experiment.name)

In [ ]:
from azureml.core.dataset import Dataset

small_dataset = Dataset.get_by_name(ws, name='oj_data_small')
small_dataset_input = small_dataset.as_named_input('train_10_models')

#dataset = Dataset.get_by_name(ws, name='oj_data')
#dataset_input = dataset.as_named_input('train_all_models')

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

train_env = Environment(name="many_models_environment")
train_conda_deps = CondaDependencies.create(pip_packages=['sklearn', 'pmdarima'])
train_env.python.conda_dependencies = train_conda_deps

In [ ]:
from azureml.core.compute import AmlCompute

compute = AmlCompute(ws, "train-many-model")

In [ ]:
from azureml.contrib.pipeline.steps import ParallelRunConfig

worker_count = 8
node_count = 5
timeout = 500

datasetname='stores_filedatasets'

tags = {}
tags['DatasetName'] = datasetname
tags['Nodes'] = node_count
tags['WorkersPerNode'] = worker_count
tags['Timeout'] = timeout

parallel_run_config = ParallelRunConfig(
    source_directory='./scripts',
    entry_script='train.py',
    mini_batch_size="1",
    run_invocation_timeout=timeout,
    error_threshold=10,
    output_action="append_row",
    environment=train_env,
    process_count_per_node=worker_count,
    compute_target=compute,
    node_count=node_count)

In [ ]:
from azureml.pipeline.core import PipelineData

output_dir = PipelineData(name="training_output", 
                          datastore=dstore)

In [ ]:
from azureml.contrib.pipeline.steps import ParallelRunStep

parallel_run_step = ParallelRunStep(
    name="many-models-training",
    parallel_run_config=parallel_run_config,
    inputs=[small_dataset_input], # train 10 models
#     inputs=[dataset_input], # switch to this inputs if train all 11,973 models
    output=output_dir,
    models=[],
    arguments=['--target_column', 'Quantity', 
               '--n_test_periods', 6, 
               '--timestamp_column', 'WeekStarting', 
               '--stepwise_training', True])

### Create the PythonScriptStep

We then set up a PythonScriptStep to retrieve the output of the training pipeline (in this case, a file with trained models' logging information) and upload it to a dedicated blob path.

### Set up RunConfiguration for PythonScriptStep

Run configuration represents configuration for experiment runs targeting different compute targets in Azure Machine Learning. The RunConfiguration object encapsulates the information necessary to submit a training run in an experiment. Here we define azureml-pipeline-core and Pandas packages.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import CondaDependencies

run_config = RunConfiguration(framework="python")
run_config.target = compute
cd = CondaDependencies.create(pip_packages=['azureml-pipeline-core'], conda_packages=['pandas'])
run_config.environment.python.conda_dependencies = cd

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

script_step = PythonScriptStep(name="logging",
                        script_name="log.py",
                        compute_target=compute,
                        source_directory='./scripts',
                        runconfig=run_config,
                        arguments=['--parallelrunstep_name', parallel_run_step.name,
                                   '--datastore', dstore.name, 
                                   '--experiment', experiment.name, 
                                   '--overwrite_logs', True, 
                                   '--pipeline_output_name', output_dir.name])

In [ ]:
from azureml.pipeline.core import StepSequence

training_steps = StepSequence(steps=[parallel_run_step, script_step])

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline = Pipeline(workspace=ws, steps=training_steps)
run = experiment.submit(pipeline,tags=tags)
RunDetails(run).show()

In [ ]:
# run.wait_for_completion(show_output=True)

In [ ]:
published_pipeline = pipeline.publish(name = 'train_many_models',
                                     description = 'train many models and log the run',
                                     version = '1',
                                     continue_on_step_failure = False)

In [ ]:
from azureml.pipeline.core import Schedule, ScheduleRecurrence
    
training_pipeline_id = published_pipeline.id

recurrence = ScheduleRecurrence(frequency="Month", interval=1, start_time="2020-01-01T09:00:00")
recurring_schedule = Schedule.create(ws, name="training_pipeline_recurring_schedule", 
                            description="Schedule Training Pipeline to run on the first day of every month starting Jan 1, 2020 at 9AM",
                            pipeline_id=training_pipeline_id, 
                            experiment_name=experiment.name, 
                            recurrence=recurrence)